<a href="https://colab.research.google.com/github/Prakriti-Sahu/FoML/blob/main/assign-4_q6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Q6. New York Taxi-cab Fare-Prediction**

**Installing Kaggle dependencies**

In [3]:
!pip install kaggle
!mkdir .kaggle

**Importing data**

In [5]:
import json
token = {"username":"prakritisahu","key":"c8dce2c6b383533a84d1ddc341353420"}
with open('/content/.kaggle/kaggle.json', 'w') as file:
    json.dump(token, file)

!cp /content/.kaggle/kaggle.json ~/.kaggle/kaggle.json

!kaggle config set -n path -v{/content}

!chmod 600 /root/.kaggle/kaggle.json

!kaggle competitions download -c new-york-city-taxi-fare-prediction

!unzip /content/{/content}/competitions/new-york-city-taxi-fare-prediction/train.csv.zip && rm /content/{/content}/competitions/new-york-city-taxi-fare-prediction/train.csv.zip

- path is now set to: {/content}
  0% 0.00/335k [00:00<?, ?B/s]
100% 335k/335k [00:00<00:00, 49.7MB/s]
  0% 0.00/486 [00:00<?, ?B/s]
100% 486/486 [00:00<00:00, 485kB/s]
  0% 0.00/960k [00:00<?, ?B/s]
100% 960k/960k [00:00<00:00, 48.8MB/s]
100% 1.55G/1.56G [00:40<00:00, 50.5MB/s]
100% 1.56G/1.56G [00:40<00:00, 41.4MB/s]
Archive:  /content/{/content}/competitions/new-york-city-taxi-fare-prediction/train.csv.zip
  inflating: train.csv               


**Importing necessary libraries**

In [6]:
import pandas as pd
import sklearn
from sklearn.preprocessing import LabelEncoder
import numpy as np
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import cross_val_score
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
import math
from sklearn import preprocessing
from sklearn.metrics import mean_squared_error

**Reading train and test data**

In [228]:
train_data=pd.read_csv("train.csv", nrows=1000000)
test_data=pd.read_csv("/content/{/content}/competitions/new-york-city-taxi-fare-prediction/test.csv")

## **Data Preprocessing and Feature Selection**

In [229]:
train_data.describe()

,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
count,1000000.000000,1000000.000000,1000000.000000,999990.000000,999990.000000,1000000.000000
mean,11.348079,-72.526640,39.929008,-72.527860,39.919954,1.684924
std,9.822090,12.057937,7.626154,11.324494,8.201418,1.323911
min,-44.900000,-3377.680935,-3116.285383,-3383.296608,-3114.338567,0.000000
25%,6.000000,-73.992060,40.734965,-73.991385,40.734046,1.000000
50%,8.500000,-73.981792,40.752695,-73.980135,40.753166,1.000000
75%,12.500000,-73.967094,40.767154,-73.963654,40.768129,2.000000
max,500.000000,2522.271325,2621.628430,45.581619,1651.553433,208.000000


**Removing outliers**

In [230]:
def train_remove_outliers(df):
    return df[(df['fare_amount'] >= 1) & 
              (df['fare_amount'] <= 500) &
              (df['pickup_longitude'] >= -75) & 
              (df['pickup_longitude'] <= -72) & 
              (df['dropoff_longitude'] >= -75) & 
              (df['dropoff_longitude'] <= -72) & 
              (df['pickup_latitude'] >= 40) & 
              (df['pickup_latitude'] <= 42) & 
              (df['dropoff_latitude'] >=40) & 
              (df['dropoff_latitude'] <= 42) & 
              (df['passenger_count'] >= 1) & 
              (df['passenger_count'] <= 6)]


def test_remove_outliers(df):
    return df[(df['pickup_longitude'] >= -75) & 
              (df['pickup_longitude'] <= -72) & 
              (df['dropoff_longitude'] >= -75) & 
              (df['dropoff_longitude'] <= -72) & 
              (df['pickup_latitude'] >= 40) & 
              (df['pickup_latitude'] <= 42) & 
              (df['dropoff_latitude'] >=40) & 
              (df['dropoff_latitude'] <= 42) & 
              (df['passenger_count'] >= 1) & 
              (df['passenger_count'] <= 6)]

**Splitting pickup_datetime**

In [231]:
def date_split(data, c):
  data[c]=pd.to_datetime(data[c])
  data['year'] = data[c].dt.year
  data['month'] = data[c].dt.month
  data['day'] = data[c].dt.day
  data['weekday'] = data[c].dt.weekday
  data[c + '_hour'] = data[c].dt.hour

In [233]:
def removal(data):

  upper=np.percentile(data,[99.6])[0]
  print(upper)
  for i in range(data.shape[0]):
    if(data.iloc[i]>upper):
      data.iloc[i]=np.nan

**Calculating Haversine distance for an estimate of distance travelled by a cab**

In [234]:
def distance(pickup_latitude, pickup_longitude, dropoff_latitude, dropoff_longitude):
    radius = 6371
    pickup_latitude, pickup_longitude, dropoff_latitude, dropoff_longitude = map(np.radians,[pickup_latitude, pickup_longitude, dropoff_latitude, dropoff_longitude])
    distance_latitude = dropoff_latitude - pickup_latitude
    distance_longitude = dropoff_longitude - pickup_longitude
    calculation = np.sin(distance_latitude/2.0)**2 + np.cos(pickup_latitude) * np.cos(dropoff_latitude) * np.sin(distance_longitude/2.0)**2
    
    return 2 * radius * np.arcsin(np.sqrt(calculation))

train_data['distance'] = distance(train_data['pickup_latitude'], train_data['pickup_longitude'], train_data['dropoff_latitude'], train_data['dropoff_longitude'])
#test_data['distance'] = distance(test_data['pickup_latitude'], test_data['pickup_longitude'], test_data['dropoff_latitude'], test_data['dropoff_longitude'])

In [235]:
train_data.shape

(1000000, 9)

**Removing outliers and NaN values, that might have occured**

In [236]:
col = train_data.columns

In [237]:
train_data = train_remove_outliers(train_data)

In [238]:
train_data.isnull().sum().sum()

0

In [239]:
removal(train_data['distance'])
removal(train_data['fare_amount'])
train_data.dropna(inplace=True)

21.48563762192481


/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


57.33


In [240]:
date_split(train_data,'pickup_datetime')

In [241]:
train_data.shape

(968935, 14)

In [242]:
train_data.isnull().sum().sum()

0

In [243]:
train_data.drop(['key','pickup_datetime','pickup_longitude','pickup_latitude','dropoff_longitude','dropoff_latitude'],axis=1,inplace=True)

In [244]:
train_data.head()

,fare_amount,passenger_count,distance,year,month,day,weekday,pickup_datetime_hour
0,4.5,1,1.030764,2009,6,15,0,17
1,16.9,1,8.450134,2010,1,5,1,16
2,5.7,2,1.389525,2011,8,18,3,0
3,7.7,1,2.799270,2012,4,21,5,4
4,5.3,1,1.999157,2010,3,9,1,7


In [245]:
train_data.columns

Index(['fare_amount', 'passenger_count', 'distance', 'year', 'month', 'day',
       'weekday', 'pickup_datetime_hour'],
      dtype='object')

In [246]:
train_data.isnull().sum().sum()

0

In [202]:
train_data.dropna(inplace=True)

In [247]:
train_data.shape

(968935, 8)

**Splitting train data into train set and validation set**

In [248]:
train_data.columns

Index(['fare_amount', 'passenger_count', 'distance', 'year', 'month', 'day',
       'weekday', 'pickup_datetime_hour'],
      dtype='object')

In [249]:
X=train_data[['passenger_count', 'distance', 'year', 'month', 'day', 'weekday',
       'pickup_datetime_hour']]
y=train_data['fare_amount']

In [250]:
X_train, X_val, y_train, y_val = train_test_split(X,y, test_size=0.3, random_state=42)

In [251]:
from sklearn.preprocessing import StandardScaler

scaler=StandardScaler().fit(X_train)
scaler_2=StandardScaler().fit(X_val)

train_scaled_data=scaler.transform(X_train)
test_scaled_data=scaler_2.transform(X_val)

In [252]:
col = X_train.columns

In [253]:
df=pd.DataFrame(train_scaled_data,columns=col)
print(df.head())
dt=pd.DataFrame(test_scaled_data,columns=col)
print(dt.head())

   passenger_count  distance  ...   weekday  pickup_datetime_hour
0        -0.528873 -0.519530  ... -1.558885              1.456962
1         0.237230 -0.332905  ...  1.518378             -2.073874
2         2.535539 -0.518996  ...  1.005501             -1.613330
3        -0.528873 -0.471187  ...  1.518378             -1.613330
4        -0.528873 -0.189938  ... -1.558885              0.842904

[5 rows x 7 columns]
   passenger_count  distance  ...   weekday  pickup_datetime_hour
0        -0.528987  0.228821  ... -1.560477              0.073832
1         3.290806 -0.488449  ... -0.534002             -1.154278
2         0.234971  2.008806  ...  1.005712             -0.233196
3        -0.528987 -0.695292  ...  0.492474              0.227346
4        -0.528987  0.156615  ... -0.534002             -0.233196

[5 rows x 7 columns]


## **Training**

**Gradient Boosting Regressor**

In [255]:
from sklearn.ensemble import GradientBoostingRegressor

gbr = GradientBoostingRegressor(random_state=1)
gbr.fit(X_train, y_train)
pred_gbr = gbr.predict(X_train)

print('RMSE train:',mean_squared_error(y_train, pred_gbr,squared=False))
print('RMSE validation:',mean_squared_error(y_val, gbr.predict(X_val), squared=False))

RMSE train: 3.5783996062869705
RMSE validation: 3.5648068512010114


**Random Forest**

In [257]:
rf = RandomForestRegressor(random_state=1)
rf.fit(X_train, y_train)
pred_rf = rf.predict(X_train)

print('RMSE train:',mean_squared_error(y_train, pred_rf,squared=False))
print('RMSE validation:',mean_squared_error(y_val, rf.predict(X_val), squared=False))

RMSE train: 1.3887377459954888
RMSE validation: 3.6766991377629306


**Extreme Gradient Boosting**

In [258]:
xgboost = XGBRegressor(random_state=42)
xgboost.fit(X_train,y_train)
pred_xgboost=xgboost.predict(X_train)

print('RMSE train:', mean_squared_error(y_train, pred_xgboost,squared=False))
print('RMSE validation:', mean_squared_error(y_val, xgboost.predict(X_val),squared=False))

[16:26:25] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
RMSE train: 3.5791181785547734
RMSE validation: 3.565779939641823


**Neural Network**

In [276]:
regr = MLPRegressor(random_state=1)
regr.fit(X_train, y_train)
pred_regr = regr.predict(X_train)

print('RMSE train:', mean_squared_error(y_train, pred_regr,squared=False))
print('RMSE validation:', mean_squared_error(y_val, regr.predict(X_val),squared=False))

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:699: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


RMSE train: 3.9519636365921937
RMSE validation: 3.9220175849411745


**Decision Tree**

In [256]:
dec = DecisionTreeRegressor(random_state=1, max_depth=6)
dec.fit(X_train, y_train)
pred_dec = dec.predict(X_train)

print('RMSE train:', mean_squared_error(y_train, pred_dec,squared=False))
print('RMSE validation:', mean_squared_error(y_val, dec.predict(X_val),squared=False))

RMSE train: 0.2210640154799549
RMSE validation: 5.241048731528136


## **Testing**

In [ ]:
#test_data=pd.read_csv("/content/{/content}/competitions/new-york-city-taxi-fare-prediction/test.csv")

In [259]:
test_data.columns

Index(['key', 'pickup_datetime', 'pickup_longitude', 'pickup_latitude',
       'dropoff_longitude', 'dropoff_latitude', 'passenger_count'],
      dtype='object')

In [260]:
test_data['distance'] = distance(test_data['pickup_latitude'], test_data['pickup_longitude'], test_data['dropoff_latitude'], test_data['dropoff_longitude'])

In [261]:
test_data.isnull().sum().sum()

0

In [262]:
date_split(test_data,'pickup_datetime')

In [263]:
test_data.drop(['pickup_datetime','pickup_longitude','pickup_latitude','dropoff_longitude','dropoff_latitude'],axis=1,inplace=True)

In [266]:
test = test_data.drop(['key'], axis=1)

In [267]:
scaler3=StandardScaler().fit(test)
test_scaled=scaler_2.transform(test)

## **Submission**

**Predicting on test data and submission**

In [268]:
pred1=gbr.predict(test)

In [269]:
pred2=xgboost.predict(test)

In [277]:
pred3=rf.predict(test)

In [270]:
test_data['fare_amount']=pred1
test_data.head(5)

,key,passenger_count,distance,year,month,day,weekday,pickup_datetime_hour,fare_amount
0,2015-01-27 13:08:24.0000002,1,2.323260,2015,1,27,1,13,10.679194
1,2015-01-27 13:08:24.0000003,1,2.425353,2015,1,27,1,13,11.156049
2,2011-10-08 11:53:44.0000002,1,0.618628,2011,10,8,5,11,4.867517
3,2012-12-01 21:12:12.0000002,1,1.961033,2012,12,1,5,21,8.009054
4,2012-12-01 21:12:12.0000003,1,5.387301,2012,12,1,5,21,15.089023


In [271]:
result1=test_data[['key','fare_amount']]
result1.to_csv("submission1", index=False)

In [272]:
test_data['fare_amount']=pred2
test_data.head()

,key,passenger_count,distance,year,month,day,weekday,pickup_datetime_hour,fare_amount
0,2015-01-27 13:08:24.0000002,1,2.323260,2015,1,27,1,13,10.466498
1,2015-01-27 13:08:24.0000003,1,2.425353,2015,1,27,1,13,10.983000
2,2011-10-08 11:53:44.0000002,1,0.618628,2011,10,8,5,11,4.885765
3,2012-12-01 21:12:12.0000002,1,1.961033,2012,12,1,5,21,8.058046
4,2012-12-01 21:12:12.0000003,1,5.387301,2012,12,1,5,21,15.215985


In [275]:
result2=test_data[['key','fare_amount']]
result2.to_csv("submission2", index=False)

In [278]:
test_data['fare_amount']=pred3
test_data.head()

,key,passenger_count,distance,year,month,day,weekday,pickup_datetime_hour,fare_amount
0,2015-01-27 13:08:24.0000002,1,2.323260,2015,1,27,1,13,10.975
1,2015-01-27 13:08:24.0000003,1,2.425353,2015,1,27,1,13,11.200
2,2011-10-08 11:53:44.0000002,1,0.618628,2011,10,8,5,11,5.256
3,2012-12-01 21:12:12.0000002,1,1.961033,2012,12,1,5,21,8.001
4,2012-12-01 21:12:12.0000003,1,5.387301,2012,12,1,5,21,17.528


In [279]:
result3=test_data[['key','fare_amount']]
result3.to_csv("submission3", index=False)